In [1]:
import os
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

sc.settings.verbosity = 0
sc.settings.set_figure_params(dpi=300, dpi_save=500)

In [2]:
data_dir = '../../../Data/Spatial/Proteomics/MIBI-TOF_TNBC_Keren2018/'
cold_group = [15, 19, 22, 24, 25, 26]  # 6
mixed_group = [1, 2, 7, 8, 11, 12, 13, 14, 17, 18, 20, 21, 23, 27, 29, 31, 33, 38, 39]  # 19
compartmentalized_group = [3, 4, 5, 6, 9, 10, 16, 28, 32, 34, 35, 36, 37, 40, 41]  #15

In [ ]:
adata_list = []
for i in range(1, 42):
    if i == 30: continue
    adata = ad.read_h5ad(data_dir + f'TNBC_p{i}.h5ad')
    
    adata_new = ad.AnnData(X=adata.X.copy())
    adata_new.obs_names = adata.obs_names.copy()
    adata_new.var_names = adata.var_names.copy()
    
    adata_new.obs['SampleID'] = adata.obs['SampleID'].copy()
    adata_new.obs['cellLabelInImage'] = adata.obs['cellLabelInImage'].copy()
    adata_new.obs['cellSize'] = adata.obs['cellSize'].copy()
    adata_new.obs['tumorYN'] = adata.obs['tumorYN'].copy()
    adata_new.obs['group_name'] = adata.obs['group_name'].copy()
    adata_new.obs['immuneGroup_name'] = adata.obs['immuneGroup_name'].copy()
    adata_new.obs['all_group_name'] = adata.obs['all_group_name'].copy()

    adata_new.obsm['spatial'] = adata.obsm['spatial'].copy()

    if i in cold_group:
        adata_new.obs['sample_types'] = ['cold' for _ in range(adata.shape[0])]
    elif i in mixed_group:
        adata_new.obs['sample_types'] = ['mixed' for _ in range(adata.shape[0])]
    elif i in compartmentalized_group:
        adata_new.obs['sample_types'] = ['compartmentalized' for _ in range(adata.shape[0])]

    # adata_new.write_h5ad(data_dir + f'TNBC_p{i}.h5ad')
    adata_list.append(adata_new)

In [4]:
adata_concat = ad.concat(adata_list, label='slice_id')
adata_concat

AnnData object with n_obs × n_vars = 197678 × 36
    obs: 'SampleID', 'cellLabelInImage', 'cellSize', 'tumorYN', 'group_name', 'immuneGroup_name', 'all_group_name', 'sample_types', 'slice_id'
    obsm: 'spatial'

In [5]:
list(adata_concat.obs.all_group_name.unique())

['Keratin+ tumor',
 'CD3 T',
 'B',
 'CD4 T',
 'DC/Mono',
 'Unidentified',
 'Macrophages',
 'CD8 T',
 'Other immune',
 'Endothelial',
 'Mono/Neu',
 'Mesenchymal like',
 'Neutrophils',
 'NK',
 'Tumor',
 'DC',
 'Tregs']

In [6]:
adata_concat.obs['all_group_name'].value_counts()

all_group_name
Keratin+ tumor      99487
Macrophages         20616
CD8 T               15698
CD4 T               12438
B                    9115
Mesenchymal like     8170
Other immune         6891
DC/Mono              5049
CD3 T                3848
Tumor                3167
Mono/Neu             3110
Neutrophils          3018
Endothelial          2086
Unidentified         1725
Tregs                1341
DC                   1245
NK                    674
Name: count, dtype: int64

In [16]:
sc.pl.heatmap(adata_concat,
              var_names=adata.var_names.tolist(),  
              groupby='all_group_name',                
              show=True,
              swap_axes=False,
              cmap='RdBu_r', 
              vmax=2, 
              vmin=-2, 
              figsize=(15, 15),
              )   